In [2]:
import time
from openai import OpenAI

In [3]:



client = OpenAI(
    # This is the default and can be omitted
    api_key="sk-proj-INTJGxHzWPvrrvIzjo74T3BlbkFJ8HXGcVxcHKAT1di61JkN"
)



In [4]:
def wait_for_assistant(thread, run):
    """
        Function to periodically check run status of AI assistant and print run time
    """

    # wait for assistant process prompt
    t0 = time.time()
    while run.status != 'completed':

        # retreive status of run (this might take a few seconds or more)
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )

        # wait 0.5 seconds
        time.sleep(0.25)
    dt = time.time() - t0
    print("Elapsed time: " + str(dt) + " seconds")
    
    return run

In [10]:
intstructions_string = "MahiGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–MahiGPT'. \
MahiGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging."

assistant = client.beta.assistants.create(
    name="MahiGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string,
    model="gpt-3.5-turbo"
)
print(assistant)

Assistant(id='asst_xIGCZbiTtoECFvrVTv6HbksL', created_at=1719056766, description='Data scientist GPT for YouTube comments', instructions="MahiGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and concludes with its signature '–MahiGPT'. MahiGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.", metadata={}, model='gpt-3.5-turbo', name='MahiGPT', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)


In [11]:
thread = client.beta.threads.create()

# generate user message
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [ ]:
# wait for assistant process prompt
run = wait_for_assistant(thread, run)

# view run object (in Jupyter Lab)
dict(run)

In [ ]:
# view messages added to thread
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

In [ ]:
client.beta.assistants.delete(assistant.id)

Few-shot prompting

In [ ]:
intstructions_string_few_shot = """MahiGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–MahiGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Here are examples of ShawGPT responding to viewer comments.

Viewer comment: This was a very thorough introduction to LLMs and answered many questions I had. Thank you.
ShawGPT: Great to hear, glad it was helpful :) -MahiGPT

Viewer comment: Epic, very useful for my BCI class
ShawGPT: Thanks, glad to hear! -MahiGPT

Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Shaw. Thank you. It's so rare to find good communicators like you!
ShawGPT: Thanks, glad it was clear -MahiGPT"""

In [ ]:
assistant = client.beta.assistants.create(
    name="MahiGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string_few_shot,
    model="gpt-3.5-turbo"
)

In [ ]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a few seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

In [ ]:
# print assistant response 
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

RAG
Add docs for retrieval of information 

In [ ]:
# create file (note: this will create a presisting file for your openai account, so be mindful about how many times you run this. 
# You can delete unnecessary files in the "Files" tab of your openai account. 

file = client.files.create(
  file=open("articles/4 Ways to Quantify Fat Tails with Python _ by Shaw Talebi _ Towards Data Science.pdf", "rb"),
  purpose="assistants"
)

In [ ]:
create new assistants with access to docs

Technical question 

In [ ]:
# create new thread
thread = client.beta.threads.create()

# generate technical question
user_message = "What is fat-tailedness?"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response (this might take a several seconds or more)
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)

In [ ]:
# print assistant response 
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

In [ ]:
# delete assistant
client.beta.assistants.delete(assistant.id)